# NBA Passing Network Analysis

Interactive visualization and analysis of NBA passing networks across multiple seasons (2014-2024).

In [ ]:
import networkx as nx
import pandas as pd
from pyvis.network import Network
import math
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import numpy as np

## 1. Load Network Data

In [ ]:
# Load both networks
print("Loading networks...")
G_assist = nx.read_gexf('output/nba_assist_network.gexf')
G_pass = nx.read_gexf('output/nba_pass_network.gexf')

print(f"Assist Network: {G_assist.number_of_nodes()} nodes, {G_assist.number_of_edges()} edges")
print(f"Pass Network: {G_pass.number_of_nodes()} nodes, {G_pass.number_of_edges()} edges")

## 2. Network Statistics

In [ ]:
def analyze_network(G, name="Network"):
    print(f"\n=== {name} ===")
    print(f"Nodes: {G.number_of_nodes()}")
    print(f"Edges: {G.number_of_edges()}")
    print(f"Density: {nx.density(G):.6f}")
    
    # Degree statistics
    out_degrees = dict(G.out_degree())
    in_degrees = dict(G.in_degree())
    
    print(f"\nOut-degree stats:")
    print(f"  Mean: {np.mean(list(out_degrees.values())):.2f}")
    print(f"  Max: {max(out_degrees.values())}")
    print(f"  Min: {min(out_degrees.values())}")
    
    # Top 10 nodes by out-degree
    top_out = sorted(out_degrees.items(), key=lambda x: x[1], reverse=True)[:10]
    print(f"\nTop 10 by out-degree (most passes/assists):")
    for i, (node, deg) in enumerate(top_out, 1):
        print(f"  {i}. {node}: {deg}")

analyze_network(G_assist, "Assist Network")
analyze_network(G_pass, "Pass Network")

## 3. Interactive Visualization Function

In [ ]:
def visualize_network(G, output_file='network.html', height='800px'):
    """
    Create an interactive visualization with improved node sizing and edge labels.
    """
    # Create PyVis network
    net = Network(height=height, width='100%', bgcolor='#222222', 
                  font_color='white', directed=True, notebook=True)
    
    # Calculate layout
    print("Calculating layout...")
    pos = nx.spring_layout(G, k=0.5, iterations=50, seed=42)
    
    # Calculate out-degrees for sizing
    out_degrees = dict(G.out_degree())
    max_out_degree = max(out_degrees.values()) if out_degrees else 1
    
    # Node sizing parameters
    min_size = 5
    max_size = 100
    scale = 1000
    
    # Apply node properties
    print("Setting node properties...")
    for node, coords in pos.items():
        G.nodes[node]['x'] = coords[0] * scale
        G.nodes[node]['y'] = coords[1] * scale
        
        # Logarithmic size scaling
        out_deg = out_degrees.get(node, 0)
        if out_deg > 0:
            log_deg = math.log(1 + out_deg)
            max_log = math.log(1 + max_out_degree)
            node_size = min_size + (log_deg / max_log) * (max_size - min_size)
        else:
            node_size = min_size
        
        G.nodes[node]['size'] = node_size
        G.nodes[node]['label'] = node.split(',')[0]  # Player name only
        G.nodes[node]['title'] = f"{node}\nOut-degree: {out_deg}"
    
    # Add edge labels
    print("Setting edge properties...")
    for source, target, data in G.edges(data=True):
        weight = data.get('weight', 1)
        G[source][target]['label'] = str(int(weight))
        G[source][target]['title'] = f"Weight: {weight}"
    
    net.from_nx(G)
    net.toggle_physics(False)
    
    # Set visualization options
    net.set_options('''
    {
      "nodes": {
        "font": {"size": 14, "strokeWidth": 2, "strokeColor": "#222222"},
        "scaling": {"label": {"enabled": true, "min": 10, "max": 30}}
      },
      "edges": {
        "arrows": {"to": {"enabled": true, "scaleFactor": 0.5}},
        "color": {"inherit": "from", "opacity": 0.6},
        "smooth": {"enabled": true, "type": "curvedCW", "roundness": 0.1},
        "width": 1,
        "font": {"size": 10, "align": "middle", "background": "rgba(0,0,0,0.7)"}
      },
      "interaction": {
        "hover": true,
        "tooltipDelay": 100,
        "zoomView": true,
        "dragView": true,
        "hideEdgesOnDrag": true,
        "hideEdgesOnZoom": true
      }
    }
    ''')
    
    print(f"Saving to {output_file}...")
    net.save_graph(output_file)
    return net

## 4. Filter Network by Season or Player

In [ ]:
def filter_by_season(G, season):
    """
    Filter network to only include nodes from a specific season.
    """
    nodes_in_season = [n for n in G.nodes() if season in n]
    return G.subgraph(nodes_in_season).copy()

def filter_by_player(G, player_name):
    """
    Filter network to show a specific player and their connections across all seasons.
    """
    # Find all nodes for this player (across all seasons)
    player_nodes = [n for n in G.nodes() if n.startswith(player_name + ',')]
    
    # Get all neighbors
    all_nodes = set(player_nodes)
    for node in player_nodes:
        all_nodes.update(G.predecessors(node))
        all_nodes.update(G.successors(node))
    
    return G.subgraph(all_nodes).copy()

# Example: Get available seasons
seasons = set()
for node in G_pass.nodes():
    parts = node.split(', ')
    if len(parts) >= 3:
        seasons.add(parts[2])

print("Available seasons:", sorted(seasons))

## 5. Visualize Full Networks

**Warning**: This may take a few minutes for large networks!

In [ ]:
# Visualize Assist Network
net_assist = visualize_network(G_assist, 'assist_network.html')
display(HTML('assist_network.html'))

In [ ]:
# Visualize Pass Network
net_pass = visualize_network(G_pass, 'pass_network.html')
display(HTML('pass_network.html'))

## 6. Filter and Visualize by Season

Pick a season to explore:

In [ ]:
# Change this to any season you want
selected_season = '2023-24'

G_season = filter_by_season(G_pass, selected_season)
print(f"Season {selected_season}: {G_season.number_of_nodes()} nodes, {G_season.number_of_edges()} edges")

net_season = visualize_network(G_season, f'season_{selected_season}.html', height='700px')
display(HTML(f'season_{selected_season}.html'))

## 7. Filter and Visualize by Player

Explore a specific player's network across all seasons:

In [ ]:
# Change this to any player name
player_name = 'LeBron James'

G_player = filter_by_player(G_pass, player_name)
print(f"Player {player_name}: {G_player.number_of_nodes()} nodes, {G_player.number_of_edges()} edges")

net_player = visualize_network(G_player, f'player_{player_name.replace(" ", "_")}.html', height='700px')
display(HTML(f'player_{player_name.replace(" ", "_")}.html'))

## 8. Degree Distribution Analysis

In [ ]:
# Plot degree distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for idx, (G, name) in enumerate([(G_assist, 'Assist'), (G_pass, 'Pass')]):
    out_degrees = [d for n, d in G.out_degree()]
    
    axes[idx].hist(out_degrees, bins=50, edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel('Out-Degree')
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'{name} Network - Out-Degree Distribution')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()